# Decomp community

In [71]:
# qiime imports
import qiime2
from qiime2 import Artifact, Metadata

# General Tool Imports
import numpy as np
import pandas as pd
import collections
from pickle import load, dump
from IPython.display import display
import warnings

# Plotting Imports
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import seaborn as sns
from statannotations.Annotator import Annotator

import itertools
import scipy
import skbio

from skbio.stats import subsample_counts
from skbio import OrdinationResults
from statsmodels.sandbox.stats.multicomp import multipletests

%matplotlib inline

In [72]:
samples = Metadata.load('11204_20230201-070843.txt').to_dataframe()

In [73]:
reads = pd.read_csv('reads_per_samples_morgue.csv', index_col=0)

In [74]:
# get samples with >1000 reads
samples = samples[samples.index.isin(reads[reads['0']>1000].index)]

In [75]:
samples['env_package'].value_counts(dropna=False).to_frame()

,env_package
built environment,108
human-skin,96


In [77]:
# drop environment samples
samples = samples.loc[samples['env_package']=='human-skin'].copy()

In [78]:
samples.loc[samples['env_package'].str.contains('skin'),'live_dead'] = 'dead'
samples.loc[samples['swab_description'].str.contains('live'),'live_dead'] = 'live'


In [79]:
samples['live_dead'].value_counts(dropna=False).to_frame()

,live_dead
dead,73
live,23


In [80]:
data = Artifact.load('table_11204.qza').view(pd.DataFrame)

In [81]:
# only get data for samples > 1000 reads
data = data[data.index.isin(samples.index)]

In [82]:
# make ra table
ra = data.apply(lambda x: x / x.sum(), axis=1)

### Import Decomposers

In [83]:
pmi_decom = pd.read_csv('../ASVs_repseq.txt',sep ='\t',index_col=3)

In [84]:
pmi_decom.head()

,#OTU ID,taxonomy,SUM,100_bp,kingdom,phylum,class,order,family,genus,species
asv,,,,,,,,,,,
TACAGAGGGTGCGAGCGTTAATCGGATTTACTGGGCGTAAAGCGTGCGTAGGCGGCTTTTTAAGTCGGATGTGAAATCCCCGAGCTTAACTTGGGAATTGCATTCGATACTGGGAAGCTAGAGTATGGGAGAGGATGGTAGAATTCCAGG,9acc238746a1f2aa7745a0b5720c4eac,D_0__Bacteria;D_1__Proteobacteria;D_2__Gammapr...,694120,TACAGAGGGTGCGAGCGTTAATCGGATTTACTGGGCGTAAAGCGTG...,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Moraxellaceae,Acinetobacter,NaN
TACAGAGGGTGCGAGCGTTAATCGGATTTACTGGGCGTAAAGCGTGCGTAGGCGGCTTTTTAAGTCGGATGTGAAATCCCTGAGCTTAACTTAGGAATTGCATTCGATACTGGGAAGCTAGAGTATGGGAGAGGATGGTAGAATTCCAGG,6e8986f8088b452f964e7968bb8bca87,D_0__Bacteria;D_1__Proteobacteria;D_2__Gammapr...,372201,TACAGAGGGTGCGAGCGTTAATCGGATTTACTGGGCGTAAAGCGTG...,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Moraxellaceae,Acinetobacter,NaN
TACAGAGGGTGCAAGCGTTAATCGGATTTACTGGGCGTAAAGCGTACGTAGGCGGCTAATTAAGTCGGATGTGAAATCCCCGAGCTCAACTTGGGAATTGCATTCGATACTGGTTAGCTAGAGTATGGGAGAGGATGGTAGAATTCCAGG,9e33b8985d44f2ed1b88ac7079f70793,D_0__Bacteria;D_1__Proteobacteria;D_2__Gammapr...,143184,TACAGAGGGTGCAAGCGTTAATCGGATTTACTGGGCGTAAAGCGTA...,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Moraxellaceae,Acinetobacter,NaN
TACAGAGGGTGCGAGCGTTAATCGGATTTACTGGGCGTAAAGCGTACGTAGGCGGCTTTTTAAGTCGGATGTGAAATCCCTGAGCTTAACTTAGGAATTGCATTCGATACTGGGAAGCTAGAGTATGGGAGAGGATGGTAGAATTCCAGG,d5902b1353d3aa0effaaa81cba1d1516,D_0__Bacteria;D_1__Proteobacteria;D_2__Gammapr...,108334,TACAGAGGGTGCGAGCGTTAATCGGATTTACTGGGCGTAAAGCGTA...,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Moraxellaceae,Acinetobacter,NaN
TACAGAGGGTGCAAGCGTTAATCGGATTTACTGGGCGTAAAGCGCGCGTAGGCGGCTAATTAAGTCAAATGTGAAATCCCCGAGCTTAACTTGGGAATTGCATTCGATACTGGTTAGCTAGAGTGTGGGAGAGGATGGTAGAATTCCAGG,1ba09da532a0f9985b3caf72a47b42e3,D_0__Bacteria;D_1__Proteobacteria;D_2__Gammapr...,101710,TACAGAGGGTGCAAGCGTTAATCGGATTTACTGGGCGTAAAGCGCG...,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Moraxellaceae,Acinetobacter,NaN


In [85]:
pmi_decom.shape

(35, 11)

In [86]:
# pmi decomposers found in fly data
morgue_decomp = list(set(pmi_decom.index.tolist()).intersection(set(data.columns.tolist())))

In [87]:
len(morgue_decomp)

29

In [88]:
# group samples
ra_morgue = ra[morgue_decomp].groupby(samples['live_dead']).mean()

In [89]:
ra_morgue.T.groupby(pmi_decom['genus']).sum()

live_dead,dead,live
genus,,
Acinetobacter,0.005800,0.009610
Bacteroides,0.000685,0.000109
Ignatzschineria,0.007188,0.000627
Oblitimonas,0.000608,0.000113
Peptoniphilus,0.003536,0.003788
Savagea,0.006669,0.000041
Vagococcus,0.000357,0.000167
Wohlfahrtiimonas,0.000227,0.000000


### how many samples are these decomposer asvs found in?

In [90]:
# of samples each asv is found in
pd.concat([pmi_decom.loc[pmi_decom.index.isin(morgue_decomp)][['genus','species']],
           data[morgue_decomp].groupby(samples['live_dead']).apply(lambda x: (x>0).sum()).T],axis=1)

,genus,species,dead,live
TACAGAGGGTGCGAGCGTTAATCGGATTTACTGGGCGTAAAGCGTGCGTAGGCGGCTTTTTAAGTCGGATGTGAAATCCCCGAGCTTAACTTGGGAATTGCATTCGATACTGGGAAGCTAGAGTATGGGAGAGGATGGTAGAATTCCAGG,Acinetobacter,NaN,50,6
TACAGAGGGTGCGAGCGTTAATCGGATTTACTGGGCGTAAAGCGTGCGTAGGCGGCTTTTTAAGTCGGATGTGAAATCCCTGAGCTTAACTTAGGAATTGCATTCGATACTGGGAAGCTAGAGTATGGGAGAGGATGGTAGAATTCCAGG,Acinetobacter,NaN,28,10
TACAGAGGGTGCGAGCGTTAATCGGATTTACTGGGCGTAAAGCGTACGTAGGCGGCTTTTTAAGTCGGATGTGAAATCCCTGAGCTTAACTTAGGAATTGCATTCGATACTGGGAAGCTAGAGTATGGGAGAGGATGGTAGAATTCCAGG,Acinetobacter,NaN,1,0
TACAGAGGGTGCAAGCGTTAATCGGATTTACTGGGCGTAAAGCGCGCGTAGGCGGCTAATTAAGTCAAATGTGAAATCCCCGAGCTTAACTTGGGAATTGCATTCGATACTGGTTAGCTAGAGTGTGGGAGAGGATGGTAGAATTCCAGG,Acinetobacter,NaN,43,13
TACGTAGGTGGCAAGCGTTGTCCGGAATTATTGGGCGTAAAGCGCACGCAGGCGGCCTTCTAAGTCTGATGTGAAATCCCACGGCTTAACCGTGGAAGGTCATTGGAAACTGGAAGGCTTGAGGATAGAAGAGGAAAGTGGAATTCCACG,Savagea,uncultured bacterium,35,1
TACGTAGGTGGCAAGCGTTGTCCGGAATTATTGGGCGTAAAGCGCACGCAGGCGGCCTTTTAAGTCTGATGTGAAATCCCACGGCTTAACCGTGGAAGGTCATTGGAAACTGGAAGGCTTGAGGATAGAAGAGGAAAGTGGAATTCCACG,Savagea,uncultured bacterium,3,0
TACGAAGGGTGCAAGCGTTAATCGGAATTACTGGGCGTAAAGCGCGCGTAGGTGGTTTGTTAAGTTGGAAGTGAAAGCCCCGGGCTCAACCTGGGAATTGCTTTCAAAACTAGCAGGCTAGAGTACAGTAGAGGGTAGTGGAATTTCCTG,Oblitimonas,Oblitimonas alkaliphila,17,1
TACGTAGGTGGCAAGCGTTGTCCGGATTTATTGGGCGTAAAGCGAGCGCAGGCGGTCTTTTAAGTCTGATGTGAAAGCCCTCGGCTCAACCGAGGAAGGTCATTGGAAACTGGGAGACTTGAGTGCAGAAGAGGAGAGTGGAATTCCATG,Vagococcus,NaN,11,0
TACGTAGGTGGCAAGCGTTGTCCGGATTTATTGGGCGTAAAGCGAGCGCAGGCGGTCTTTTAAGTCTGATGTGAAAGCCCTCGGCTCAACCGAGGAAGGTCATTGGAAACTGGAGGACTTGAGTGCAGAAGAGGAGAGTGGAATTCCATG,Vagococcus,NaN,6,2
TACGTAGGTGGCAAGCGTTGTCCGGATTTATTGGGCGTAAAGCGAGCGCAGGCGGTTTCTTAAGTCTGATGTGAAAGCCCTCGGCTCAACCGAGGAAGGTCATTGGAAACTGGGGAACTTGAGTGCAGAAGAGGAGAGTGGAATTCCATG,Vagococcus,NaN,13,0


In [91]:
## percent of samples each ASV is found in
pd.concat([pmi_decom.loc[pmi_decom.index.isin(morgue_decomp)][['genus','species']], 
           (data[morgue_decomp].groupby(samples['live_dead']).apply(
    lambda x: (x>0).sum()).T)/(data[morgue_decomp].groupby(samples['live_dead']).count().T)*100], axis=1)

,genus,species,dead,live
TACAGAGGGTGCGAGCGTTAATCGGATTTACTGGGCGTAAAGCGTGCGTAGGCGGCTTTTTAAGTCGGATGTGAAATCCCCGAGCTTAACTTGGGAATTGCATTCGATACTGGGAAGCTAGAGTATGGGAGAGGATGGTAGAATTCCAGG,Acinetobacter,NaN,68.493151,26.086957
TACAGAGGGTGCGAGCGTTAATCGGATTTACTGGGCGTAAAGCGTGCGTAGGCGGCTTTTTAAGTCGGATGTGAAATCCCTGAGCTTAACTTAGGAATTGCATTCGATACTGGGAAGCTAGAGTATGGGAGAGGATGGTAGAATTCCAGG,Acinetobacter,NaN,38.356164,43.478261
TACAGAGGGTGCGAGCGTTAATCGGATTTACTGGGCGTAAAGCGTACGTAGGCGGCTTTTTAAGTCGGATGTGAAATCCCTGAGCTTAACTTAGGAATTGCATTCGATACTGGGAAGCTAGAGTATGGGAGAGGATGGTAGAATTCCAGG,Acinetobacter,NaN,1.369863,0.000000
TACAGAGGGTGCAAGCGTTAATCGGATTTACTGGGCGTAAAGCGCGCGTAGGCGGCTAATTAAGTCAAATGTGAAATCCCCGAGCTTAACTTGGGAATTGCATTCGATACTGGTTAGCTAGAGTGTGGGAGAGGATGGTAGAATTCCAGG,Acinetobacter,NaN,58.904110,56.521739
TACGTAGGTGGCAAGCGTTGTCCGGAATTATTGGGCGTAAAGCGCACGCAGGCGGCCTTCTAAGTCTGATGTGAAATCCCACGGCTTAACCGTGGAAGGTCATTGGAAACTGGAAGGCTTGAGGATAGAAGAGGAAAGTGGAATTCCACG,Savagea,uncultured bacterium,47.945205,4.347826
TACGTAGGTGGCAAGCGTTGTCCGGAATTATTGGGCGTAAAGCGCACGCAGGCGGCCTTTTAAGTCTGATGTGAAATCCCACGGCTTAACCGTGGAAGGTCATTGGAAACTGGAAGGCTTGAGGATAGAAGAGGAAAGTGGAATTCCACG,Savagea,uncultured bacterium,4.109589,0.000000
TACGAAGGGTGCAAGCGTTAATCGGAATTACTGGGCGTAAAGCGCGCGTAGGTGGTTTGTTAAGTTGGAAGTGAAAGCCCCGGGCTCAACCTGGGAATTGCTTTCAAAACTAGCAGGCTAGAGTACAGTAGAGGGTAGTGGAATTTCCTG,Oblitimonas,Oblitimonas alkaliphila,23.287671,4.347826
TACGTAGGTGGCAAGCGTTGTCCGGATTTATTGGGCGTAAAGCGAGCGCAGGCGGTCTTTTAAGTCTGATGTGAAAGCCCTCGGCTCAACCGAGGAAGGTCATTGGAAACTGGGAGACTTGAGTGCAGAAGAGGAGAGTGGAATTCCATG,Vagococcus,NaN,15.068493,0.000000
TACGTAGGTGGCAAGCGTTGTCCGGATTTATTGGGCGTAAAGCGAGCGCAGGCGGTCTTTTAAGTCTGATGTGAAAGCCCTCGGCTCAACCGAGGAAGGTCATTGGAAACTGGAGGACTTGAGTGCAGAAGAGGAGAGTGGAATTCCATG,Vagococcus,NaN,8.219178,8.695652
TACGTAGGTGGCAAGCGTTGTCCGGATTTATTGGGCGTAAAGCGAGCGCAGGCGGTTTCTTAAGTCTGATGTGAAAGCCCTCGGCTCAACCGAGGAAGGTCATTGGAAACTGGGGAACTTGAGTGCAGAAGAGGAGAGTGGAATTCCATG,Vagococcus,NaN,17.808219,0.000000
